In [ ]:
#Check the PyTorch and Cuda version
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"

1.10.0+cu111
11.1


In [ ]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install torch-cluster -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html

In [2]:
#First data characteristics
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root='/tmp/CiteSeer', name='CiteSeer',transform=NormalizeFeatures())

data = dataset[0]
print(data)

print(f'Number of nodes: {data.num_nodes}')
print(f'Nodes features: {data.num_node_features}')
print(f'Number of classes: {dataset.num_classes}')
print(f'Number of edges: {data.num_edges}')
print(f'Avarage degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Training nodes: {data.train_mask.sum()}')
print(f'Validation nodes: {data.val_mask.sum()}')
print(f'Test nodes: {data.test_mask.sum()}')
print(f'Isolated nodes: {data.has_isolated_nodes()}')
print(f'Loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Processing...


Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])
Number of nodes: 3327
Nodes features: 3703
Number of classes: 6
Number of edges: 9104
Avarage degree: 2.74
Training nodes: 120
Validation nodes: 500
Test nodes: 1000
Isolated nodes: True
Loops: False
Is undirected: True


Done!


**Applying self-attention to all nodes using the first 1500 nodes to train**

Using GNNs, the prediction of each node is done using neural network layers on an aggregation of neighboring nodes. Now we are going to use the building block of a Transformer, applying self-attention to all nodes of the graph. A part of these nodes will be used for training and another for validation and test.

In [3]:
#Add a new batch dimension of size 1 to the tensor
import torch

data1=torch.unsqueeze(data.x, 1)
data1.shape

torch.Size([3327, 1, 3703])

In [4]:
#Convert the input features dimension to the model features
#Apply self-attention using PyTorch MultiheadAttention
#Transform the encoder output to the output dimension
#Set up the model and tensors in cuda
import torch.nn
from torch.nn import Linear
import torch.nn.functional as F

class Self_Att(torch.nn.Module):
    def __init__(self):
        super(Self_Att, self).__init__()
        torch.manual_seed(12345)
        self.lin1 = Linear(3703,120)
        self.attention1 = torch.nn.MultiheadAttention(120,6)
        self.lin2 = Linear(120, dataset.num_classes)

    def forward(self,x):
        x = self.lin1(x)
        x = F.dropout(x, p=0.8, training=self.training)
        x, attn_output_weights = self.attention1(x,x,x)
        x = x.relu()
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin2(x)
        x=torch.squeeze(x)#Remove the dimension of size 1
        return x,attn_output_weights

use_cuda = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

model = Self_Att()
model.cuda()

data=data.cuda()
data1=data1.cuda()

print(model)

Using device: cuda
Self_Att(
  (lin1): Linear(in_features=3703, out_features=120, bias=True)
  (attention1): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=120, out_features=120, bias=True)
  )
  (lin2): Linear(in_features=120, out_features=6, bias=True)
)


In [9]:
#For training, the cross entropy loss combines LogSoftmax and NLLLoss in one single class
#input is expected to contain raw, unnormalized scores for each class
#target a class index in the range for each value of a 1D tensor of size minibatch
#For testing, compares the class with highest probability with the labels and counts the correct predictions
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)  # Define optimizer.

def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out,attn_output_weights = model(data1)  # Perform a single forward pass.
      loss = criterion(out[0:1500], data.y[0:1500])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss,attn_output_weights

def test():
      model.eval()
      out,attn_output_weights = model(data1)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc,attn_output_weights

for epoch in range(1, 201):
    loss,attn_output_weights = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
    #print(attn_output_weights)

<IPython.core.display.Javascript object>

Epoch: 001, Loss: 1.7458
Epoch: 002, Loss: 1.7432
Epoch: 003, Loss: 1.7429
Epoch: 004, Loss: 1.7431
Epoch: 005, Loss: 1.7432
Epoch: 006, Loss: 1.7429
Epoch: 007, Loss: 1.7424
Epoch: 008, Loss: 1.7431
Epoch: 009, Loss: 1.7424
Epoch: 010, Loss: 1.7435
Epoch: 011, Loss: 1.7428
Epoch: 012, Loss: 1.7427
Epoch: 013, Loss: 1.7428
Epoch: 014, Loss: 1.7422
Epoch: 015, Loss: 1.7427
Epoch: 016, Loss: 1.7428
Epoch: 017, Loss: 1.7423
Epoch: 018, Loss: 1.7427
Epoch: 019, Loss: 1.7425
Epoch: 020, Loss: 1.7431
Epoch: 021, Loss: 1.7424
Epoch: 022, Loss: 1.7425
Epoch: 023, Loss: 1.7428
Epoch: 024, Loss: 1.7427
Epoch: 025, Loss: 1.7425
Epoch: 026, Loss: 1.7427
Epoch: 027, Loss: 1.7427
Epoch: 028, Loss: 1.7425
Epoch: 029, Loss: 1.7424
Epoch: 030, Loss: 1.7434
Epoch: 031, Loss: 1.7430
Epoch: 032, Loss: 1.7433
Epoch: 033, Loss: 1.7424
Epoch: 034, Loss: 1.7429
Epoch: 035, Loss: 1.7427
Epoch: 036, Loss: 1.7429
Epoch: 037, Loss: 1.7427
Epoch: 038, Loss: 1.7426
Epoch: 039, Loss: 1.7427
Epoch: 040, Loss: 1.7431


In [10]:
test_acc,attn_output_weights = test()
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.2310


In [ ]:
attn_output_weights[0]

tensor([[0.0003, 0.0003, 0.0003,  ..., 0.0003, 0.0003, 0.0003],
        [0.0003, 0.0003, 0.0003,  ..., 0.0003, 0.0003, 0.0003],
        [0.0003, 0.0003, 0.0003,  ..., 0.0003, 0.0003, 0.0003],
        ...,
        [0.0003, 0.0003, 0.0003,  ..., 0.0003, 0.0003, 0.0003],
        [0.0003, 0.0003, 0.0003,  ..., 0.0003, 0.0003, 0.0003],
        [0.0003, 0.0003, 0.0003,  ..., 0.0003, 0.0003, 0.0003]],
       device='cuda:0', grad_fn=<SelectBackward0>)

Self-attention to all nodes with the first 1500 nodes to train, accuracy 23.10%.


**Applying self-attention with an attention mask (only self-loops) using the first 1500 nodes to train**

In [11]:
#Creating an attention mask with all ones but 0 in the diagonal
att_mask=torch.ones(3327,3327)
att_mask.fill_diagonal_(0)
att_mask=att_mask.bool()
att_mask

tensor([[False,  True,  True,  ...,  True,  True,  True],
        [ True, False,  True,  ...,  True,  True,  True],
        [ True,  True, False,  ...,  True,  True,  True],
        ...,
        [ True,  True,  True,  ..., False,  True,  True],
        [ True,  True,  True,  ...,  True, False,  True],
        [ True,  True,  True,  ...,  True,  True, False]])

In [12]:
#Convert the input features dimension to the model features
#Apply self-attention using PyTorch MultiheadAttention
#Transform the encoder output to the output dimension
#Set up the model and tensors in cuda
import torch.nn
from torch.nn import Linear
import torch.nn.functional as F

class Self_Att(torch.nn.Module):
    def __init__(self):
        super(Self_Att, self).__init__()
        torch.manual_seed(12345)
        self.lin1 = Linear(3703,120)
        self.attention1 = torch.nn.MultiheadAttention(120,6)
        self.lin2 = Linear(120, dataset.num_classes)

    def forward(self,x,att_mask):
        x = self.lin1(x)
        x = F.dropout(x, p=0.8, training=self.training)
        x, attn_output_weights = self.attention1(x,x,x,attn_mask=att_mask)
        x = x.relu()
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin2(x)
        x=torch.squeeze(x)#Remove the dimension of size 1
        return x,attn_output_weights


use_cuda = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

model = Self_Att()
model.cuda()

data=data.cuda()
data1=data1.cuda()
att_mask=att_mask.cuda()

Using device: cuda


In [13]:
#For training, the cross entropy loss combines LogSoftmax and NLLLoss in one single class
#input is expected to contain raw, unnormalized scores for each class
#target a class index in the range for each value of a 1D tensor of size minibatch
#For testing, compares the class with highest probability with the labels and counts the correct predictions
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)  # Define optimizer.

def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out,attn_output_weights = model(data1,att_mask)  # Perform a single forward pass.
      loss = criterion(out[0:1500], data.y[0:1500])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss,attn_output_weights

def test():
      model.eval()
      out,attn_output_weights = model(data1,att_mask)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc,attn_output_weights

for epoch in range(1, 201):
    loss,attn_output_weights = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
    #print(attn_output_weights)

<IPython.core.display.Javascript object>

Epoch: 001, Loss: 1.7803
Epoch: 002, Loss: 1.7700
Epoch: 003, Loss: 1.7512
Epoch: 004, Loss: 1.7445
Epoch: 005, Loss: 1.7364
Epoch: 006, Loss: 1.7306
Epoch: 007, Loss: 1.7262
Epoch: 008, Loss: 1.7153
Epoch: 009, Loss: 1.6874
Epoch: 010, Loss: 1.6451
Epoch: 011, Loss: 1.5767
Epoch: 012, Loss: 1.4859
Epoch: 013, Loss: 1.3884
Epoch: 014, Loss: 1.3093
Epoch: 015, Loss: 1.2276
Epoch: 016, Loss: 1.1319
Epoch: 017, Loss: 1.0653
Epoch: 018, Loss: 0.9622
Epoch: 019, Loss: 0.8921
Epoch: 020, Loss: 0.8197
Epoch: 021, Loss: 0.7359
Epoch: 022, Loss: 0.6766
Epoch: 023, Loss: 0.5958
Epoch: 024, Loss: 0.5371
Epoch: 025, Loss: 0.4690
Epoch: 026, Loss: 0.4280
Epoch: 027, Loss: 0.3744
Epoch: 028, Loss: 0.3375
Epoch: 029, Loss: 0.2867
Epoch: 030, Loss: 0.2415
Epoch: 031, Loss: 0.2159
Epoch: 032, Loss: 0.1864
Epoch: 033, Loss: 0.1777
Epoch: 034, Loss: 0.1403
Epoch: 035, Loss: 0.1457
Epoch: 036, Loss: 0.1427
Epoch: 037, Loss: 0.1161
Epoch: 038, Loss: 0.1082
Epoch: 039, Loss: 0.0917
Epoch: 040, Loss: 0.0990


In [14]:
test_acc,attn_output_weights = test()
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.6910


In [15]:
attn_output_weights

tensor([[[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.]]], device='cuda:0',
       grad_fn=<DivBackward0>)

Self-attention to only the same node with the first 1500 nodes to train, accuracy 69.10%.

**Using Transformers with an attention mask (representing the graph structure) using the first 1500 nodes to train**

In [16]:
import numpy as np
import networkx as nx
from torch_geometric.utils import to_networkx

G = to_networkx(data, to_undirected=False)
att_mask=nx.to_numpy_matrix(G)
np.fill_diagonal(att_mask,1)

zero_indices = att_mask == 0
non_zero_indices = att_mask != 0
att_mask[non_zero_indices] = 0
att_mask[zero_indices] = 1

att_mask=torch.from_numpy(att_mask)
att_mask=att_mask.bool()
att_mask

tensor([[False,  True,  True,  ...,  True,  True,  True],
        [ True, False,  True,  ...,  True,  True,  True],
        [ True,  True, False,  ...,  True,  True,  True],
        ...,
        [ True,  True,  True,  ..., False,  True,  True],
        [ True,  True,  True,  ...,  True, False,  True],
        [ True,  True,  True,  ...,  True,  True, False]])

In [17]:
#Convert the input features dimension to the model features
#Apply self-attention using PyTorch MultiheadAttention
#Transform the encoder output to the output dimension
#Set up the model and tensors in cuda
import torch.nn
from torch.nn import Linear
import torch.nn.functional as F

class Self_Att(torch.nn.Module):
    def __init__(self):
        super(Self_Att, self).__init__()
        torch.manual_seed(12345)
        self.lin1 = Linear(3703,120)
        self.attention1 = torch.nn.MultiheadAttention(120,6)
        self.lin2 = Linear(120, dataset.num_classes)

    def forward(self,x,att_mask):
        x = self.lin1(x)
        x = F.dropout(x, p=0.8, training=self.training)
        x, attn_output_weights = self.attention1(x,x,x,attn_mask=att_mask)
        x = x.relu()
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin2(x)
        x=torch.squeeze(x)#Remove the dimension of size 1
        return x,attn_output_weights


use_cuda = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

model = Self_Att()
model.cuda()

data=data.cuda()
data1=data1.cuda()
att_mask=att_mask.cuda()

Using device: cuda


In [18]:
#For training, the cross entropy loss combines LogSoftmax and NLLLoss in one single class
#input is expected to contain raw, unnormalized scores for each class
#target a class index in the range for each value of a 1D tensor of size minibatch
#For testing, compares the class with highest probability with the labels and counts the correct predictions
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)  # Define optimizer.

def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out,attn_output_weights = model(data1,att_mask)  # Perform a single forward pass.
      loss = criterion(out[0:1500], data.y[0:1500])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss,attn_output_weights

def test():
      model.eval()
      out,attn_output_weights = model(data1,att_mask)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc,attn_output_weights

for epoch in range(1, 201):
    loss,attn_output_weights = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
    #print(attn_output_weights)

<IPython.core.display.Javascript object>

Epoch: 001, Loss: 1.7802
Epoch: 002, Loss: 1.7703
Epoch: 003, Loss: 1.7507
Epoch: 004, Loss: 1.7453
Epoch: 005, Loss: 1.7379
Epoch: 006, Loss: 1.7326
Epoch: 007, Loss: 1.7294
Epoch: 008, Loss: 1.7234
Epoch: 009, Loss: 1.7043
Epoch: 010, Loss: 1.6766
Epoch: 011, Loss: 1.6234
Epoch: 012, Loss: 1.5507
Epoch: 013, Loss: 1.4606
Epoch: 014, Loss: 1.3573
Epoch: 015, Loss: 1.2662
Epoch: 016, Loss: 1.1814
Epoch: 017, Loss: 1.0849
Epoch: 018, Loss: 1.0176
Epoch: 019, Loss: 0.9325
Epoch: 020, Loss: 0.8770
Epoch: 021, Loss: 0.8124
Epoch: 022, Loss: 0.7600
Epoch: 023, Loss: 0.7068
Epoch: 024, Loss: 0.6577
Epoch: 025, Loss: 0.5970
Epoch: 026, Loss: 0.5467
Epoch: 027, Loss: 0.5210
Epoch: 028, Loss: 0.4806
Epoch: 029, Loss: 0.4479
Epoch: 030, Loss: 0.4468
Epoch: 031, Loss: 0.4443
Epoch: 032, Loss: 0.3636
Epoch: 033, Loss: 0.3701
Epoch: 034, Loss: 0.3505
Epoch: 035, Loss: 0.3562
Epoch: 036, Loss: 0.2985
Epoch: 037, Loss: 0.3411
Epoch: 038, Loss: 0.2853
Epoch: 039, Loss: 0.2715
Epoch: 040, Loss: 0.2810


In [19]:
test_acc,attn_output_weights = test()
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.7300


In [20]:
data.edge_index[:,0:10]

tensor([[   0,    1,    1,    1,    1,    1,    2,    3,    3,    4],
        [ 628,  158,  486, 1097, 2919, 2933, 3285, 1431, 3219,  467]],
       device='cuda:0')

In [21]:
print(attn_output_weights)

tensor([[[0.5000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.1667, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.5000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.2500, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.5000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.5000]]],
       device='cuda:0', grad_fn=<DivBackward0>)


In [22]:
def model_summary(model):
    
    model_params_list = list(model.named_parameters())
    print("----------------------------------------------------------------")
    line_new = "{:>20}  {:>25} {:>15}".format("Layer.Parameter", "Param Tensor Shape", "Param #")
    print(line_new)
    print("----------------------------------------------------------------")
    for elem in model_params_list:
        p_name = elem[0] 
        p_shape = list(elem[1].size())
        p_count = torch.tensor(elem[1].size()).prod().item()
        line_new = "{:>20}  {:>25} {:>15}".format(p_name, str(p_shape), str(p_count))
        print(line_new)
    print("----------------------------------------------------------------")
    total_params = sum([param.nelement() for param in model.parameters()])
    print("Total params:", total_params)
    num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Trainable params:", num_trainable_params)
    print("Non-trainable params:", total_params - num_trainable_params)

model_summary(model)

----------------------------------------------------------------
     Layer.Parameter         Param Tensor Shape         Param #
----------------------------------------------------------------
         lin1.weight                [120, 3703]          444360
           lin1.bias                      [120]             120
attention1.in_proj_weight                 [360, 120]           43200
attention1.in_proj_bias                      [360]             360
attention1.out_proj.weight                 [120, 120]           14400
attention1.out_proj.bias                      [120]             120
         lin2.weight                   [6, 120]             720
           lin2.bias                        [6]               6
----------------------------------------------------------------
Total params: 503286
Trainable params: 503286
Non-trainable params: 0


Self-attention to the neighboring nodes with the first 1500 nodes to train, accuracy 73%.

**The basic model but in Cuda for comparisions**

In [ ]:
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv



class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(dataset.num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

model = GCN(hidden_channels=16)

model.cuda()
data=data.cuda()


In [ ]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data.x,data.edge_index)  # Perform a single forward pass.
      loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test():
      model.eval()
      out = model(data.x,data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc


for epoch in range(1, 201):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Loss: 1.7915
Epoch: 002, Loss: 1.7875
Epoch: 003, Loss: 1.7823
Epoch: 004, Loss: 1.7752
Epoch: 005, Loss: 1.7683
Epoch: 006, Loss: 1.7605
Epoch: 007, Loss: 1.7525
Epoch: 008, Loss: 1.7423
Epoch: 009, Loss: 1.7387
Epoch: 010, Loss: 1.7282
Epoch: 011, Loss: 1.7168
Epoch: 012, Loss: 1.7133
Epoch: 013, Loss: 1.6932
Epoch: 014, Loss: 1.6868
Epoch: 015, Loss: 1.6759
Epoch: 016, Loss: 1.6629
Epoch: 017, Loss: 1.6535
Epoch: 018, Loss: 1.6446
Epoch: 019, Loss: 1.6305
Epoch: 020, Loss: 1.6317
Epoch: 021, Loss: 1.6121
Epoch: 022, Loss: 1.6036
Epoch: 023, Loss: 1.5931
Epoch: 024, Loss: 1.5798
Epoch: 025, Loss: 1.5575
Epoch: 026, Loss: 1.5395
Epoch: 027, Loss: 1.5365
Epoch: 028, Loss: 1.5098
Epoch: 029, Loss: 1.4833
Epoch: 030, Loss: 1.4816
Epoch: 031, Loss: 1.4661
Epoch: 032, Loss: 1.4568
Epoch: 033, Loss: 1.4259
Epoch: 034, Loss: 1.4174
Epoch: 035, Loss: 1.4302
Epoch: 036, Loss: 1.4050
Epoch: 037, Loss: 1.3624
Epoch: 038, Loss: 1.3600
Epoch: 039, Loss: 1.3464
Epoch: 040, Loss: 1.2972


In [ ]:
test_acc = test()
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.7090


In [ ]:
def model_summary(model):
    
    model_params_list = list(model.named_parameters())
    print("----------------------------------------------------------------")
    line_new = "{:>20}  {:>25} {:>15}".format("Layer.Parameter", "Param Tensor Shape", "Param #")
    print(line_new)
    print("----------------------------------------------------------------")
    for elem in model_params_list:
        p_name = elem[0] 
        p_shape = list(elem[1].size())
        p_count = torch.tensor(elem[1].size()).prod().item()
        line_new = "{:>20}  {:>25} {:>15}".format(p_name, str(p_shape), str(p_count))
        print(line_new)
    print("----------------------------------------------------------------")
    total_params = sum([param.nelement() for param in model.parameters()])
    print("Total params:", total_params)
    num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Trainable params:", num_trainable_params)
    print("Non-trainable params:", total_params - num_trainable_params)

model_summary(model)

----------------------------------------------------------------
     Layer.Parameter         Param Tensor Shape         Param #
----------------------------------------------------------------
          conv1.bias                       [16]              16
    conv1.lin.weight                 [16, 3703]           59248
          conv2.bias                        [6]               6
    conv2.lin.weight                    [6, 16]              96
----------------------------------------------------------------
Total params: 59366
Trainable params: 59366
Non-trainable params: 0
